---
layout: post
title: DataGeo
subtitle: Dados Espaciais
tags: [python, jupyter, package, datageo, gis]
image: /img/posts/datageo_icon.png
bigimg: /img/posts/datageo_big.png
gh-repo: michelmetran/geo_SP_DataGeo
gh-badge: [follow, star, watch, fork]
comments: true

---

<br>

# *Imports* e Funções

Inicialmente faz-se necessário importar as bibliotecas que serão necessárias.

In [ ]:
import os
import re
import shutil
import zipfile
import requests
import geopandas as gpd
from datetime import date
from bs4 import BeautifulSoup

<br>

Após isso cria-se as pastas listadas, que armazenarão as informações ao longo desse *script*.

In [ ]:
os.makedirs(os.path.join('data', 'brutos'), exist_ok=True)

<br>

Usei a função abaixo para fazer *download* usando o *request*. Ainda, a função pega o nome do arquivo a partir do *Content Disposition*. Peguei a função do *post* [*Downloading Files from URLs in Python*](https://www.codementor.io/@aviaryan/downloading-files-from-urls-in-python-77q3bs0un), aonde tem outros exemplos, com outras finalidades.

In [ ]:
def get_filename_from_cd(cd):
    """
    Get filename from content-disposition
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]

<br>

Após isso a forma de obtenção dos dados:
1. Acessar a página dos metadados do plano de informação e exportar, mantendo armazenadas as informações da origem desse material;
2. Caso seja possível acessar o material cartográfica por *shapefile*, será possível tomar conhecimento disso na página dos metadados;
3. Uma função específica procura o *link* do *shapefile* e faz o download, extração da pasta zipada.
4. Promove-se correções na tabela de atributos e nas projeções geográficas.

In [ ]:
# Input dos caminhos para os metadados
url = 'http://datageo.ambiente.sp.gov.br/geoportal/catalog/search/resource/details.page?uuid='
id_metadados = '{74040682-561A-40B8-BB2F-E188B58088C1}'

# Resultados
url_meta = '{}{}'.format(url, id_metadados)
print('Página com metadados:\n{}'.format(url_meta))

In [ ]:
# Abre a página dos metadados
page = requests.get(url_meta)
print('Resposta da página foi {}'.format(page))

# Parser HTML
soup = BeautifulSoup(page.content, 'html.parser')
soup = soup.find_all('a', href=True)

# Procura Shapefile
for i in soup:
    text = i.text.split(' ')
    for j in text:
        if j in 'Shapefile':
            print('> Encontrei o shapefile')
            url = i['href']
            print('Link: {}'.format(url))

In [ ]:
# Download do arquivo e paga o nome a partir do content-disposition
# Arquivo zip (shapefile) vai para a pasta de 'data/brutos'
r = requests.get(url, allow_redirects=True)
filename = get_filename_from_cd(r.headers.get('content-disposition'))
open(os.path.join('data', 'brutos', filename), 'wb').write(r.content)

In [ ]:
# Com o nome do arquivo, é realizado o download da página dos metadados
file_meta = filename.split('.')[0]
r = requests.get(url_meta, allow_redirects=True)
open(os.path.join('data', 'brutos', '{}.html'.format(file_meta)), 'wb').write(r.content)

In [ ]:
# Unzip
file = os.path.join('data', 'brutos', filename)
temp = os.path.join(os.path.dirname(file), 'temp')
os.makedirs(temp, exist_ok=True)

with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall(temp)

In [ ]:
# Lista Arquivos
os.listdir(temp)

In [ ]:
# Pega o nome do shapefile PRECISA HAVER SOMENTE UM!
shp = [i for i in os.listdir(temp) if i.endswith('.shp')]
shp[0]

In [ ]:
# Read shapefile
gdf = gpd.read_file(os.path.join(temp, shp[0]))
display(gdf.head(5))
gdf.plot()

In [ ]:
# Reprojeta
print(gdf.crs)
gdf = gdf.to_crs(epsg=4326)
print(gdf.crs)
gdf.plot()

In [ ]:
# Excluí pasta temporária
shutil.rmtree(temp)

<br>

# Função

Uma vez criada uma sequencia de códigos, foi possível definir uma funão que integra todos eles, apresentada abaixo:

In [ ]:
def download_datageo_shp(url_meta):
    print('Página com metadados: {}'.format(url_meta))

    # Abre a página dos metadados
    page = requests.get(url_meta)
    print('Resposta da página foi {}'.format(page))

    # Parser HTML
    soup = BeautifulSoup(page.content, 'html.parser')
    soup = soup.find_all('a', href=True)

    # Procura Shapefile
    for i in soup:
        text = i.text.split(' ')
        #print(text)
        for j in text:
            #print(j)
            if j in 'Shapefile':
                print('> Encontrei o shapefile')
                url = i['href']
                print('Link: {}'.format(url))

    # Download do arquivo e paga o nome a partir do content-disposition
    # Arquivo zip (shapefile) vai para a pasta de 'data/brutos'
    r = requests.get(url, allow_redirects=True)
    filename = get_filename_from_cd(r.headers.get('content-disposition'))
    open(os.path.join('data', 'brutos', filename), 'wb').write(r.content)

    # Com o nome do arquivo, é realizado o download da página dos metadados
    file_meta = filename.split('.')[0]
    r = requests.get(url_meta, allow_redirects=True)
    open(os.path.join('data', 'brutos', '{}.html'.format(file_meta)), 'wb').write(r.content)

    # Unzip
    file = os.path.join('data', 'brutos', filename)
    temp = os.path.join(os.path.dirname(file), 'temp')
    os.makedirs(temp, exist_ok=True)
    with zipfile.ZipFile(file, 'r') as zip_ref: zip_ref.extractall(temp)

    # Lista Arquivos
    os.listdir(temp)

    # Pega o nome do shapefile PRECISA HAVER SOMENTE UM!
    shp = [i for i in os.listdir(temp) if i.endswith('.shp')]
    a = len(shp)
    b = shp[0]
    print('Encontrei {} arquivos ".shp", sendo que o primeiro deles é o "{}"'.format(a, b))

    # Read shapefile
    gdf = gpd.read_file(os.path.join(temp, shp[0]))
    print(gdf.head(5))

    # Reprojeta
    print(gdf.crs)
    gdf = gdf.to_crs(epsg=4326)
    print(gdf.crs)
    gdf.plot()

    # Excluí pasta temporária
    shutil.rmtree(temp)
    
    return gdf

In [ ]:
download_datageo_shp(url_meta)

___

# Encerramento

In [ ]:
try:
    if __name__ == '__main__':
        main()
except:
    pass

In [ ]:
import os
from traitlets.config import Config
from nbconvert import PythonExporter
from nbconvert.preprocessors import TagRemovePreprocessor

In [ ]:
# Input
filename = 'get_data_datageo.ipynb'
notebook = os.path.join(os.getcwd(), filename)

In [ ]:
# Import the exporter
c = Config()
c.TagRemovePreprocessor.enabled=True
c.ClearOutputPreprocessor.enabled=True
c.TemplateExporter.exclude_markdown=True
c.TemplateExporter.exclude_code_cell=False
c.TemplateExporter.exclude_input_prompt=True
c.TemplateExporter.exclude_output=True
c.TemplateExporter.exclude_raw=True
c.TagRemovePreprocessor.remove_cell_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_cell',)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_cell',)
c.preprocessors = ['TagRemovePreprocessor']
c.PythonExporter.preprocessors = ['nbconvert.preprocessors.TagRemovePreprocessor']

# Configure and run out exporter
py_exporter = PythonExporter(config=c)
py_exporter.register_preprocessor(TagRemovePreprocessor(config=c), True)

# Configure and run out exporter - returns a tuple - first element with html, second with notebook metadata
body, metadata = PythonExporter(config=c).from_filename(notebook)

# Write to output html file
with open('{}.py'.format(filename.replace('.ipynb', '')),  'w') as f:
    f.write(body)